In [ ]:
import os
os.chdir("../..")

In [ ]:

from openai import OpenAI
import json
from app.settings import Settings
from playwright.async_api import async_playwright
import re
from urllib.parse import urlparse

In [ ]:
client = OpenAI(api_key=Settings.OPENAI_API_KEY)
MODEL = "gpt-4o-mini"

In [ ]:
async def visit_webpage(url: str) -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        parsed_url = urlparse(url)
        domain = parsed_url.netloc if parsed_url.netloc else url 
        match = re.search(r"www\.(.+?)\.com", domain)
        if match:
            subdomain = match.group(1)
            screenshot_path = f"screenshot_{subdomain}.png"
        else:
            screenshot_path = "screenshot_default.png"
        
        await page.goto(f"https://{url}")
        await page.wait_for_load_state("networkidle")

        await page.screenshot(path=screenshot_path)
        content = await page.content()
        await browser.close()
        return f"Página visitada. Captura: {screenshot_path}. HTML: {len(content)} caracteres."

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "visit_webpage",
            "description": "Visitar una pagina web especifica",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "La url de la pagina a visitar. Ej: www.test.com"
                    }
                },
                "required": ["url"]
            }
        }
    }
]


In [ ]:
assistant = client.beta.assistants.create(
    name="Asistente Personalizado",
    instructions="Eres un asistente útil que puede usar herramientas. Mantén conversaciones naturales y amigables.",
    tools=tools,
    model=MODEL
)

In [ ]:
async def handle_tool_call(tool_call: dict[str, Any]) -> str:
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    if function_name == "visit_webpage":
        return await visit_webpage(url=arguments['url'])
    
    return "Función no implementada"

In [ ]:
async def chat_loop():
    thread = client.beta.threads.create()
    
    print("¡Hola! Soy tu asistente de IA. Escribe 'exit' para terminar.\n")
    
    while True:
        user_input = input("Tú: ")
        
        if user_input.lower() == 'exit':
            print("¡Hasta luego!")
            break
        
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=user_input
        )
        
        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions="Responde de manera natural como humano."
        )
        
        while True:
            run_status = client.beta.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
            
            if run_status.status == 'completed':
                messages = client.beta.threads.messages.list(
                    thread_id=thread.id
                )
                
                last_message = messages.data[0].content[0].text.value
                print(f"\nAsistente: {last_message}\n")
                break
                
            elif run_status.status == 'requires_action':
                tool_outputs = []
                for tool_call in run_status.required_action.submit_tool_outputs.tool_calls:
                    output = await handle_tool_call(tool_call)
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": output
                    })
                
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

In [ ]:
await chat_loop()